# Dataset description
* 32 participants [s01 to s32]
* 40 pieces of music
* 40 channels out of which the first 32 are relevant 
* 8064 readings recorded at 128Hz


In [1]:
import datetime

In [2]:
from deap_pre_process import *

In [38]:
eeg1d,eeg2d,val,aro = get_subject_data(subject_no=31,media_no=[1])

[+] Reading subject: s31 Media: [1]


In [39]:
len(eeg2d)

1

In [20]:
eeg2d[0]

array([[[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.97634296,  0.        ,  0.27024688, ...,  1.83158521,
          0.        ,  0.90841254],
        ..., 
        [-0.84192473,  0.        , -1.73728569, ...,  0.74493385,
          0.        , -0.07862335],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ]],

       [[ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        , ...,  0.        ,
          0.        ,  0.        ],
        [-1.70241285,  0.        , -0.99594524, ...,  0.1825022 ,
          0.        , -1.09179632],
        ..., 
        [-1.94723405,  0.        ,  0.55361742, ...,

In [41]:
aro[0]

1

In [26]:
val

[1, 1, 1, 1, 1]

# Making the dummy network


In [27]:
from rnn_function import *
from cnn_function import *
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 4570331269983473878
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 1477437030
locality {
  bus_id: 1
  links {
  }
}
incarnation: 10566350469649338050
physical_device_desc: "device: 0, name: GeForce 930MX, pci bus id: 0000:01:00.0, compute capability: 5.0"
]


In [28]:
#Constant :
WINDOW_SIZE = 128
#For rnn layers
INPUT_NODES = 32
HIDDEN_ST = 32
HOLD_PROBA = 0.5
RNN_INPUT_SIZE = 32

#For cnn layers
CONV_IP_SHAPE = [-1,9,9,1]
CONV1_KERNEL = [4,4,1,32]
CONV2_KERNEL = [4,4,32,64]
CONV3_KERNEL = [4,4,64,128]
REDUCTION_KERNEL = [1,1,128*WINDOW_SIZE,13]

#For dnn layers (for the paper code)
N_FC_IN = 1024
N_FC_OUT= 1024

#FOR PREDICTIONS
N_LABELS = 2
LEARNING_RATE = 0.0001

In [29]:
#FUNCTION FORM https://github.com/ynulonger/ijcnn/blob/master/cv.py
#FOR SHAPING THE INPUT OF LSTM
def apply_fully_connect(x, x_size, fc_size):
    fc_weight = init_weight([x_size, fc_size])
    fc_bias = init_bias([fc_size])
    return tf.nn.elu(tf.add(tf.matmul(x, fc_weight), fc_bias))

#FOR READOUT OF THE FINAL LAYER
def apply_readout(x, x_size, readout_size):
    readout_weight = init_weight([x_size, readout_size])
    readout_bias = init_bias([readout_size])
    return tf.add(tf.matmul(x, readout_weight), readout_bias)

In [30]:
#Resetting tf default graph
tf.reset_default_graph()

#CNN placeholders and model variables
x_data_cnn = tf.placeholder(dtype = tf.float32,shape=(None,9,9),name='cnn_data')
x_image = tf.reshape(x_data_cnn,CONV_IP_SHAPE,name='cnn_image')

#LAYERS
conv1 = conv_layer(x_image,CONV1_KERNEL,name="convo_l1")
conv2 = conv_layer(conv1,CONV2_KERNEL,name="convo_l2")
conv3 = conv_layer(conv2,CONV3_KERNEL,name="convo_l3")

#Depth variable
depth = tf.reshape(conv3,[-1,9,9,128* WINDOW_SIZE])

#Size reduction conv layer
reduced_data = conv_layer(depth,REDUCTION_KERNEL,name="conv_reduced")

#Flattening convolution layer
shape = reduced_data.get_shape().as_list()
final_flat = tf.reshape(reduced_data, [-1, shape[1] * shape[2] * shape[3]])
cnn_out_fuse = final_flat

#RNN placeholder and model variables
x_data_rnn = tf.placeholder(dtype = tf.float32,shape=(None,WINDOW_SIZE,INPUT_NODES),name='rnn_data') #TIME_STEP = WINDOW_SIZE
keep_prob = tf.placeholder(dtype=tf.float32)

#Shaping the RNN input
shape = x_data_rnn.get_shape().as_list()
rnn_in_flat = tf.reshape(x_data_rnn, [-1, shape[2]]) #[batch_size*n_time_step, n_electrode]
rnn_fc_in = apply_fully_connect(rnn_in_flat, shape[2], N_FC_IN) #[batch_size*n_time_step, n_electrode]
lstm_in = tf.reshape(rnn_fc_in, [-1,WINDOW_SIZE,N_FC_IN]) #[batch_size, n_time_step, n_fc_in]

#Making cells
cell1 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cell2 = dropout_wrapper(keep_prob, create_LSTM_cell(HIDDEN_ST))
cells = [cell1,cell2]
final_cell = create_Stacked_cell(cells)

#Laying out the network
output, states = create_RNN(final_cell,lstm_in)

#Unstacking the output
output = tf.unstack(tf.transpose(output, [1, 0, 2]), name='lstm_out')

#Selecting the final output form the layer
rnn_output = output[-1] #[batch, fc_size]

#Shaping the output
lstm_fc_out = dnn_layer(rnn_output,N_FC_OUT) #[batch_size, n_fc_out]
lstm_fc_drop = tf.nn.dropout(lstm_fc_out, keep_prob)

#Placeholder for true values
y_true = tf.placeholder(dtype=tf.float32,shape=(None,N_LABELS)) #[Valance, Arousal]

Instructions for updating:
This class is deprecated, please use tf.nn.rnn_cell.LSTMCell, which supports all the feature this cell currently has. Please replace the existing code with tf.nn.rnn_cell.LSTMCell(name='basic_lstm_cell').


In [31]:
#Flattening the layer
final_connected_layer = tf.concat([cnn_out_fuse, lstm_fc_drop], axis=1)
shape = final_connected_layer.get_shape().as_list()
print("\nfuse_cnn_rnn:", shape)


fuse_cnn_rnn: [None, 2077]


In [32]:
#Creating output variables 
y_ = apply_readout(final_connected_layer, shape[1], N_LABELS)
y_pred = tf.argmax(tf.nn.softmax(y_), 1, name="y_pred")
y_posi = tf.nn.softmax(y_, name="y_posi")

#Making cost functions
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=y_, labels=y_true), name='loss')
train = tf.train.AdamOptimizer(LEARNING_RATE).minimize(cost)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [36]:
# Saver and variable initializer
init = tf.global_variables_initializer()
saver = tf.train.Saver()

# REMAINING:
* Session

In [37]:
eeg2d.shape

AttributeError: 'list' object has no attribute 'shape'

In [30]:
eeg1d.shape

(2, 7680, 32)

In [52]:
with tf.Session() as sess:
    sess.run(init)
    for i in range(2):
        sub = np.random.randint(1,33)
        print("SUBJECT: ",i)
        for j in range(0,40):
            eeg1d,eeg2d,val,aro = get_subject_data(subject_no=sub,media_no=[j])
            t1 = datetime.datetime.now()
            for k in range(0,7680,WINDOW_SIZE):
                pred = np.array([val[0],aro[0]]).reshape(1,2)
                x_rnn = eeg1d[0][k:k+WINDOW_SIZE].reshape(-1,WINDOW_SIZE,32)
                x_cnn = eeg2d[0][k:k+WINDOW_SIZE]
                sess.run(train, feed_dict = {keep_prob:0.5,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: pred})
            
            if j %1 == 0:
                print("Calculating accuracy")
                test_s = np.random.randint(1,33)
                test_m = np.random.randint(0,40,size=[10])
                test_eeg1d,test_eeg2d,test_val,test_aro = get_subject_data(subject_no=test_s,media_no=test_m)
                print("test_val", test_val)
                for med in range(10):
                    actual = np.array([test_val[med],test_aro[med]]).reshape(1,2)
                    x_rnn = test_eeg1d[med][0:WINDOW_SIZE].reshape(-1,WINDOW_SIZE,32)
                    x_cnn = test_eeg2d[med][0:WINDOW_SIZE]
                    correct = tf.equal(y_pred,tf.argmax(y_true,axis=1))
                    acc  = tf.reduce_mean(tf.cast(correct,dtype=tf.float32))
                    print("ACTUAL VALUES: ",actual,"\t Accuracy= ",sess.run(tf.nn.softmax(y_),feed_dict = {keep_prob:1,x_data_cnn:x_cnn, x_data_rnn: x_rnn, y_true: actual}))
            
                    
    
    
    
    

SUBJECT:  0
[+] Reading subject: s28 Media: [0]
Calculating accuracy
[+] Reading subject: s06 Media: [20, 29, 7, 25, 39, 22, 15, 4, 35, 39]
test_val [1, 1, 1, 1, 0, 0, 1, 1, 1, 0]
ACTUAL VALUES:  [[1 0]] 	 Accuracy=  [[ 0.00251123  0.99748874]]
ACTUAL VALUES:  [[1 0]] 	 Accuracy=  [[  9.99999523e-01   4.32564491e-07]]
ACTUAL VALUES:  [[1 0]] 	 Accuracy=  [[ 0.01291308  0.98708689]]
ACTUAL VALUES:  [[1 0]] 	 Accuracy=  [[  7.11439043e-07   9.99999285e-01]]
ACTUAL VALUES:  [[0 0]] 	 Accuracy=  [[  1.00000000e+00   2.73281548e-16]]
ACTUAL VALUES:  [[0 0]] 	 Accuracy=  [[  9.99987364e-01   1.26169534e-05]]
ACTUAL VALUES:  [[1 1]] 	 Accuracy=  [[  9.99999881e-01   1.20339010e-07]]
ACTUAL VALUES:  [[1 1]] 	 Accuracy=  [[  1.07753827e-10   1.00000000e+00]]
ACTUAL VALUES:  [[1 1]] 	 Accuracy=  [[ 0.99717939  0.0028206 ]]
ACTUAL VALUES:  [[0 0]] 	 Accuracy=  [[  1.00000000e+00   2.73281548e-16]]
[+] Reading subject: s28 Media: [1]
Calculating accuracy
[+] Reading subject: s09 Media: [25, 5, 27,

KeyboardInterrupt: 

In [70]:
print((t2 -t1).seconds," seconds",(t2 -t1).microseconds, "microseconds")

NameError: name 't2' is not defined

In [78]:
print([1,2,3])

[1, 2, 3]
